# VSX Catalog Data Analysis

In [1]:
# Load packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

We need to specify the values used to denote missing or null values, since there is a variable star type of "NA" for a type of nova that will throw off the null values in the vartype column otherwise. We also need to specify the dtype for each column to both speed up the data load and account for the mixed value types that appeared in some of the columns.

In [ ]:
df = pd.read_csv("../data/vsx_data.csv", encoding = 'unicode_escape', keep_default_na = False, 
                 na_values=['NULL','null', 'nan','NaN', ""],  dtype={'auid':str, 
                                                                     'name':str, 
                                                                     'const':str, 
                                                                     'ra':float, 
                                                                     'dec':float, 
                                                                     'varType':str,
                                                                     'maxMag':float, 
                                                                     'maxPass':str, 
                                                                     'minMag':float, 
                                                                     'minPass':str, 
                                                                     'epoch':float, 
                                                                     'novaYr':'UInt64', 
                                                                     'period':float, 
                                                                     'riseDur':float, 
                                                                     'specType':str,
                                                                     'disc':str})
df.head()

Next, I'm going to drop a few of the columns that don't contain information I plan on using in my analysis. I could have selected only the columns that I needed while reading the csv file, but since the dataset isn't that large and I'm using the majority of them I didn't do that.

In [ ]:
df.drop(['epoch', 'novaYr', 'disc'], axis=1, inplace=True)

In [ ]:
df.index.name = 'index'

## Things to look at:

1. Data cleanup, what's missing and why
3. How many stars have AUIDs assigned?
2. Breakdown by constellation
- How many stars are missing a variability type?
4. How many stars have come from which surveys or other sources?
5. Frequency of different types
6. Magnitude distribution
7. Period distribution
8. How many have spectral type, and is it available for the others?
9. Any potential correlations or areas for followup?
10. Create all-sky map of coordinate distribution?
11. Any chance for duplicate stars?

In [ ]:
df.shape[0]

In [ ]:
df.count()

In [ ]:
tot_auid = df['auid'].count() / df.shape[0] 
print(tot_auid * 100)

Only 3.8 percent of the records have an AUID! AUIDs can be requested for any star, however this implies that there is a significant part of this data set that don't have any obsevations by AAVSO observers in the photometry database. This seems unusually low, so I need to figure out if this is actually correct. Also, most stars don't have a nova year associated, but that is more reasonable since only a fraction of all variable stars could be expected to be variable due to nova activity. This is why I dropped that column.

One thing I see that is potentially interesting is that there are at least 1000 records that don't have a variable type set. Not sure if it's because it's not known or just not part of the record when it was uploaded to the database.

## Stars without a variability type assigned

In [ ]:
no_type = df[df['varType'].isnull()]
print(no_type.shape[0])

In [ ]:
no_type['maxMag'].describe()


In [ ]:
sns.histplot(data=no_type, x="maxMag", kde=True)
plt.show()

A lot of the stars without a variable type in VSX are too faint even at their maximum to be observed by a typical amateur's telescope, but there are definitely some in there well within reach.

In [ ]:
bright_unknown = no_type.loc[no_type['maxMag'] < 10]
print(bright_unknown)

## Sort data - Survey or Other Source

In [ ]:
df['source'] = ""

df['source'] = np.where(df['name'].str.contains('NSV'),
                                'NSV',df['source'])
df['source'] = np.where(df['name'].str.contains('ASAS'),
                                'ASAS',df['source'])
df['source'] = np.where(df['name'].str.contains('OGLE'),
                                'OGLE',df['source'])
df['source'] = np.where(df['name'].str.contains('CSS'),
                                'CSS',df['source'])
df['source'] = np.where(df['name'].str.contains('2MASS'),
                                '2MASS',df['source'])
df['source'] = np.where(df['name'].str.contains('USNO'),
                                'USNO',df['source'])
df['source'] = np.where(df['name'].str.contains('HAT'),
                                'HAT',df['source'])
df['source'] = np.where(df['name'].str.contains('PTF'),
                                'PTF',df['source'])
df['source'] = np.where(df['name'].str.contains('WASP'),
                                'WASP',df['source'])
df['source'] = np.where(df['name'].str.contains('GCVS'),
                                'GCVS',df['source'])
df['source'] = np.where(df['name'].str.contains('GDS'),
                                'GDS',df['source'])
df['source'] = np.where(df['name'].str.contains('KIC'),
                                'KIC',df['source'])
df['source'] = np.where(df['name'].str.contains('PNV'),
                                'PNV',df['source'])
df['source'] = np.where(df['name'].str.contains('MASTER'),
                                'MASTER',df['source'])
df['source'] = np.where(df['name'].str.contains('SWIFT'),
                                'SWIFT',df['source'])
df['source'] = np.where(df['name'].str.contains('ZTF'),
                                'ZTF',df['source'])
df['source'] = np.where(df['name'].str.contains('Gaia'),
                                'Gaia',df['source'])

In [ ]:
print(df['source'].value_counts())

## Breakdown by Constellation

In [ ]:
pd.set_option("display.max_rows", None)
df['const'].value_counts()

A lot of the top constellations (Sagittarius, Scutum, Aquila, Ophiuchus, and more) are along the direction of the Milky Way as seen from Earth, which would explain a much higher concentration of stars and star-forming regions.

The variability type has a ton of different options, since it can be any of the types defined by VSX, the surveys the data originally came from, or a combination of types if the type isn't certain. For easier analysis, I'm going to refine the types down to basic variable type categories: eclipsing, rotating, microlensing, pulsating, eruptive, cataclysmic, and x-ray.

In [ ]:
df['category'] = ""

eclipsing = ['EA', 'EB', 'EW', 'EP', 'EC', 'ED', 'ESD']
rotating = ['ACV', 'BY', 'CTTS/ROT', 'ELL', 'FKCOM', 'HB', 'LERI', 'PSR', 'ROT', 'RS', 'SXARI', 'SXARI/E', 'TTS/ROT', 'WTTS/ROT', 'NSIN ELL']
pulsating = ['ACEP', 'ACYG', 'AHB1', 'BCEP', 'BLAP', 'BXCIR', 'CEP', 'CW', 'DCEP', 'DSCT', 'DWLYN', 'GDOR', 'HADS', 'LB', 'LC', 'PPN', 'PVTEL', 'PVTELI', 'roAm', 'roAp', 'RR', 'RV', 'SPB', 'SR', 'SXPHE', 'V361HYA', 'V1093HER', 'ZZ', 'LPV', 'PULS']
eruptive = ['BE', 'cPNB', 'CTTS', 'DPV', 'DYPer', 'EXOR', 'FF', 'FSCMa', 'FUOR', 'GCAS', 'IA', 'IB', 'IN', 'INT', 'ISA', 'ISB', 'RCB', 'SDOR', 'TTS', 'UV', 'UVN', 'UXOR', 'WR', 'WTTS', 'ZZA', 'YSO', 'DIP', 'YY']
cataclysmic = ['AM', 'CBSS', 'DQ', 'IBWD', 'NA', 'NB', 'NC', 'NL', 'NR', 'SN', 'UG', 'V838MON', 'WDP', 'ZAND', 'CV', 'IBWD', 'VY']
xray = ['HMXB', 'IMXB', 'LMXB', 'BHXB', 'XB', 'XJ', 'XN', 'XP']

for index, row in df.iterrows():
     
    if any(substring in str(row['varType']) for substring in eclipsing):
        df.loc[index, 'category'] = 'eclipsing'
    elif any(substring in str(row['varType']) for substring in rotating):
        df.loc[index, 'category'] = 'rotating'
    elif any(substring in str(row['varType']) for substring in pulsating):
        df.loc[index, 'category'] = 'pulsating'
    elif any(substring in str(row['varType']) for substring in eruptive):
        df.loc[index, 'category'] = 'eruptive'
    elif any(substring in str(row['varType']) for substring in cataclysmic):
        df.loc[index, 'category'] = 'cataclysmic'
    elif any(substring in str(row['varType']) for substring in xray):
        df.loc[index, 'category'] = 'xray'
    elif str(row['varType']) == 'VAR':
        df.loc[index, 'category'] = 'unspec'
    elif str(row['varType']) == 'E':
        df.loc[index, 'category'] = 'eruptive'
    elif str(row['varType']) == 'M':
        df.loc[index, 'category'] = 'cataclysmic'
    elif str(row['varType']) == 'L':
        df.loc[index, 'category'] = 'pulsating'
    elif str(row['varType']) == 'N':
        df.loc[index, 'category'] = 'cataclysmic'
    else:
        df.loc[index, 'category'] = 'other'

In [ ]:
print(df['category'].value_counts())


In [ ]:
ax = sns.histplot(df, x='const', hue='category', 
             multiple='stack', palette='cubehelix')
ax.set_ylabel('count')
# Fix the legend so it's not on top of the bars.
legend = ax.get_legend()
legend.set_bbox_to_anchor((1, 1))

In [ ]:
#one_hot = pd.get_dummies(df['category'])
#df = df.join(one_hot)

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

Given the min/max mag and period, is there any way to predict star type?

In [ ]:
df_filtered = df[['maxMag', 'minMag', 'period', 'category']]
df_filtered['category'] = pd.factorize(df_filtered['category'])[0]

In [ ]:
#df_filtered = df[["maxMag", "minMag", "period", ]
y =  np.asarray(df_filtered.pop('category')).astype('int')

X_train, X_test, y_train, y_test = train_test_split(df_filtered, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

Let's create a basic neural net to see if any prediction can be made on star category from the minimum and maximum magnitudes, as well as the period given. One thing to take into consideration is that not all magnitudes are in the same band, some were given in visual, V, or G bands, and some were R (red) or B (blue). 

I'm not sure yet if normalizing any of the data will be useful. We should probably normalize the period since that can vary widely from a fraction of a day to over a year, but I want to try the model first to see what the effects of normalization would be.

In [ ]:
model = tf.keras.models.Sequential([ 
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dense(8, activation=tf.nn.softmax)
    ]) 

    # Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(0.001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy']) 
    


In [ ]:
    # Fit the model for 10 epochs adding the callbacks
    # and save the training history
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))